In [6]:
from glob import glob
import pandas as pd

## Batch inference byt5 -- only words

In [2]:
# only words, byt5
output_dir_inf = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/inferences/latex/'

In [7]:
inf_files = glob(output_dir_inf + '*.csv')
inf_files.sort()
for i,f in enumerate(inf_files):
    d = pd.read_csv(f)
    if i == 0: # first
        df_byt5_words = d.copy()
    else: # others
        df_byt5_words = pd.concat([df_byt5_words,d],ignore_index=True)

In [8]:
df_byt5_words.head()

,aligned sentences source,aligned sentences target,input_text,target_text,aligned sentences source types,aligned sentences target types,sentences source types,sentences target types,predicted_text
0,Thev suegested that optical {lis [rom the neu...,They suggested that optical flux from the neu...,Thev suegested that optical {lis [rom the neu...,They suggested that optical flux from the neu...,WWWW WWWWWWWWW WWWW WWWWWWW WWWW WWWW WWW WWW...,WWWW WWWWWWWWW WWWW WWWWWWW WWWW WWWW WWW WWW...,WWWW WWWWWWWWW WWWW WWWWWWW WWWW WWWW WWW WWW...,WWWW WWWWWWWWW WWWW WWWWWWW WWWW WWWW WWW WWW...,They suggested that optical flux from the neu...
1,The IRAS source would become another voung st...,The IRAS source would become another young st...,The IRAS source would become another voung st...,The IRAS source would become another young st...,WWW WWWW WWWWWW WWWWW WWWWWW WWWWWWW WWWWW WW...,WWW WWWW WWWWWW WWWWW WWWWWW WWWWWWW WWWWW WW...,WWW WWWW WWWWWW WWWWW WWWWWW WWWWWWW WWWWW WW...,WWW WWWW WWWWWW WWWWW WWWWWW WWWWWWW WWWWW WW...,The IRAS source would become another young st...
2,For Chandra. the data was culled by combining...,"For Chandra, the data was culled by combining...",For Chandra. the data was culled by combining...,"For Chandra, the data was culled by combining...",WWW WWWWWWWW WWW WWWW WWW WWWWWW WW WWWWWWWWW...,WWW WWWWWWWW WWW WWWW WWW WWWWWW WW WWWWWWWWW...,WWW WWWWWWWW WWW WWWW WWW WWWWWW WW WWWWWWWWW...,WWW WWWWWWWW WWW WWWW WWW WWWWWW WW WWWWWWWWW...,"For Chandra, the data was culled by combining..."
3,. Their upper limit corresponds to a ^^^^^^3c ...,@ Their upper limit corresponds to a $\sigma$ ...,. Their upper limit corresponds to a 3c dust m...,Their upper limit corresponds to a $\sigma$ d...,W WWWWW WWWWW WWWWW WWWWWWWWWWW WW W ^^^^^^II ...,@ WWWWW WWWWW WWWWW WWWWWWWWWWW WW W IIIIIIII ...,W WWWWW WWWWW WWWWW WWWWWWWWWWW WW W II WWWW W...,WWWWW WWWWW WWWWW WWWWWWWWWWW WW W IIIIIIII W...,Their upper limit corresponds to a 3c dust ma...
4,dust. temperatures of 35. 40. 45 and SOWKK are...,"dust@ temperatures of 35, 40, 45 and @@@K@ are...",dust. temperatures of 35. 40. 45 and SOWKK are...,"dust temperatures of 35, 40, 45 and K are abou...",WWWWW WWWWWWWWWWWW WW WWW WWW WW WWW WWWWW WWW...,WWWW@ WWWWWWWWWWWW WW WWW WWW WW WWW @@@W@ WWW...,WWWWW WWWWWWWWWWWW WW WWW WWW WW WWW WWWWW WWW...,WWWW WWWWWWWWWWWW WW WWW WWW WW WWW W WWW WWWW...,"dust temperatures of 35, 40, 45 and 89K are ab..."
